### Import

In [8]:
import random
import pandas as pd
import time
import json
import yaml
import shutil
from ultralytics import YOLO
from pathlib import Path

In [5]:
DATASET_YAML = Path("../data/silver/")  # point to your dataset.yaml
BASE_MODEL = "yolov8n-cls.pt"
EPOCHS = 50
IMG_SIZE = 512
FRACTION = 1
BATCH = 16
DEVICE = "cpu"
PROJECT_PATH = Path("../runs/classify")

CLASSES = {
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91,
}

In [ ]:
model = YOLO(BASE_MODEL)
random.seed(42)
N = 8 
subset = []
for cls in CLASSES.keys():
    cls_dir = Path(DATASET_YAML) / "val" / cls
    all_imgs = list(cls_dir.glob("*.jpg"))
    if len(all_imgs) >= N:
        subset.extend(random.sample(all_imgs, N))
    else:
        subset.extend(all_imgs)
results = model.predict(source=subset, imgsz=IMG_SIZE, device=DEVICE, save=True, verbose=False, project=PROJECT_PATH, name="food101_base_model",exist_ok=True)
for r in results:
    fname = Path(r.path).name
    pred_class = r.names[r.probs.top1]
    conf = r.probs.top1conf.item()
    print(f"{fname}: {pred_class} ({conf:.2f})")

Results saved to /workspaces/marktguru-home-assignment/runs/base_model
002620_76_pizza.jpg: pizza (0.99)
002667_76_pizza.jpg: pizza (0.86)
002673_76_pizza.jpg: pizza (1.00)
002610_76_pizza.jpg: pizza (0.72)
002675_76_pizza.jpg: pizza (0.96)
002650_76_pizza.jpg: pizza (0.98)
002577_76_pizza.jpg: pizza (0.96)
002504_76_pizza.jpg: pizza (0.99)
025088_90_spaghetti_bolognese.jpg: crayfish (0.91)
025101_90_spaghetti_bolognese.jpg: carbonara (0.38)
025187_90_spaghetti_bolognese.jpg: carbonara (0.80)
025023_90_spaghetti_bolognese.jpg: plate (0.68)
025036_90_spaghetti_bolognese.jpg: plate (0.29)
025201_90_spaghetti_bolognese.jpg: carbonara (1.00)
025011_90_spaghetti_bolognese.jpg: carbonara (1.00)
025068_90_spaghetti_bolognese.jpg: plate (0.50)
022289_91_spaghetti_carbonara.jpg: carbonara (0.95)
022254_91_spaghetti_carbonara.jpg: carbonara (1.00)
022257_91_spaghetti_carbonara.jpg: carbonara (0.99)
022410_91_spaghetti_carbonara.jpg: carbonara (1.00)
022473_91_spaghetti_carbonara.jpg: carbonara (

Pizza and spaghetti carbonara are recognized almost perfectly.
Spaghetti bolognese is missing as a class - confused often with carbonara.

### Fine-tune model with food101 data

The complete set consists of 750 images for training and 250 images for validation in each of three classes.

Eight models will be trained for each constellation of:
- two epoch values: 20 and 50,
- two images sizes: 224 (standard for Yolo) and 512 (images from Hugging Face)
- two dataset lengths: 75 and 750

In [ ]:
model = YOLO(BASE_MODEL)
#EPOCHS_LIST   = [20, 50]
EPOCHS_LIST   = [50]
#IMG_SIZE_LIST = [224,512]
IMG_SIZE_LIST = [512]
FRACTIONS     = [0.1, 1]
for epochs in EPOCHS_LIST:
    for img in IMG_SIZE_LIST:
        for frac in FRACTIONS:
            name = f"food101_e{epochs}_img{img}_frac{frac}"
            print(f"\n▶️ {name}")
            t0 = time.time()
            res = model.train(
                data=str(DATASET_YAML),
                epochs=epochs,
                imgsz=img,
                batch=BATCH,
                device=DEVICE,
                project=str(PROJECT_PATH),
                name=name,
                exist_ok=True,
                fraction=frac,
                verbose=False
            )
            runtime_min = round((time.time() - t0) / 60, 3)
            (PROJECT_PATH / name / "summary.json").write_text(
                json.dumps({"runtime_min": runtime_min}, ensure_ascii=False)
            )
            print(f"⏱️ runtime: {runtime_min} min")


▶️ food101_e50_img512_frac0.1
New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.201 🚀 Python-3.11.13 torch-2.8.0+cu128 CPU (Intel Xeon Platinum 8370C CPU @ 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/silver, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.1, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name

### Gather statistics of model training runs

In [52]:
rows = []
def pick_col(cols, candidates):
    for c in candidates:
        if c in cols: return c
    return None
for run in sorted(PROJECT_PATH.glob("food101_e*_img*_frac*")):
    args_p     = run / "args.yaml"
    results_p  = run / "results.csv"
    summary_p  = run / "summary.json"
    best_pt_p  = run / "weights" / "best.pt"
    if not results_p.exists():
        continue
    args = {}
    if args_p.exists():
        args = yaml.safe_load(args_p.read_text())
    df = pd.read_csv(results_p)
    top1_col = pick_col(df.columns, ["metrics/accuracy_top1", "top1", "val/top1"])
    if top1_col is None:
        best_top1 = float("nan")
    else:
        best_row_idx = df[top1_col].idxmax()
        best_top1 = float(df.loc[best_row_idx, top1_col])
    runtime_min = None
    if summary_p.exists():
        runtime_min = json.loads(summary_p.read_text()).get("runtime_min")
    rows.append({
        "run_dir": str(run.resolve()),
        "epochs": args.get("epochs"),
        "img_size": args.get("imgsz") or args.get("img_size"),
        "fraction": args.get("fraction"),
        "top1_acc_best": round(best_top1, 4) if best_top1 == best_top1 else None,  # NaN guard
        "runtime_min": runtime_min,
        "best_weights": str(best_pt_p.resolve()) if best_pt_p.exists() else None,
    })
stats = pd.DataFrame(rows)
stats_path = PROJECT_PATH / "experiment_stats.csv"
stats.to_csv(stats_path, index=False)

In [ ]:
stats = pd.read_csv(PROJECT_PATH / "experiment_stats.csv")
stats["model"] = stats["best_weights"].apply(lambda p: Path(p).parent.parent.name)
stats_subset = stats.drop(columns=["run_dir", "best_weights"])
(
    stats_subset.style
    .highlight_max(subset=['top1_acc_best'], color='lightgreen')
    .highlight_min(subset=['top1_acc_best'], color='lightcoral')
    .highlight_max(subset=['runtime_min'], color='lightcoral')
    .highlight_min(subset=['runtime_min'], color='lightgreen')
)


,epochs,img_size,fraction,top1_acc_best,runtime_min,model
0,20,224,0.100000,0.333300,6.230000,food101_e20_img224_frac0.1
1,20,224,1.000000,0.972000,28.919000,food101_e20_img224_frac1
2,20,512,0.100000,0.894700,28.893000,food101_e20_img512_frac0.1
3,20,512,1.000000,0.980000,158.982000,food101_e20_img512_frac1
4,50,224,0.100000,0.917300,14.628000,food101_e50_img224_frac0.1
5,50,224,1.000000,0.976000,73.864000,food101_e50_img224_frac1
6,50,512,0.100000,0.940000,71.703000,food101_e50_img512_frac0.1
7,50,512,1.000000,0.970700,nan,food101_e50_img512_frac1


### Inaccurate and long-training models

1. A model with minimum parameters `food101_e20_img224_frac0.1` shows a very poor accuracy result - `0.33`<br>
[Confusion matrix](../runs/classify/food101_e20_img224_frac0.1/confusion_matrix_normalized.png) shows that actually all the dishes were put into one class - pizza.<br>
A very low runtime should not distract here - we will not use this model and remove from futher analysis.
2. On the other side of the spectrum, there are models with the best accuracy (`> 98%`), but with also the longest runtime (`> 2 hours`):
    - `food101_e20_img512_frac1`
    - `food101_e50_img512_frac1`

    Let's remove them as well.

3. The remaining five models lie in the range `89-97%` for accurace and `14-74 minutes` for runtime, which we find acceptable.
4. Let's add for them some efficiency metric

In [ ]:
stats_filt = stats_subset[~stats_subset["model"].isin(["food101_e20_img224_frac0.1", "food101_e20_img512_frac1","food101_e50_img512_frac1"])]
stats_subset["efficiency"] = stats_subset["top1_acc_best"] / stats_subset["runtime_min"]
eff = stats_subset["efficiency"]
stats_subset["efficiency_norm"] = ((eff - eff.min()) / (eff.max() - eff.min())).round(2)
(
    stats_subset.style
    .highlight_max(subset=['top1_acc_best'], color='lightgreen')
    .highlight_min(subset=['top1_acc_best'], color='lightcoral')
    .highlight_max(subset=['runtime_min'], color='lightcoral')
    .highlight_min(subset=['runtime_min'], color='lightgreen')
    .highlight_max(subset=['efficiency_norm'], color='lightgreen')
    .highlight_min(subset=['efficiency_norm'], color='lightcoral')
)


### Impact of each parameter change on precision and performance

In [68]:
df = stats_subset.copy()

comparisons = []

# Epochs effect (control img_size + fraction)
for (img, frac), group in df.groupby(["img_size", "fraction"]):
    if {20, 50}.issubset(set(group["epochs"])):
        base = group[group["epochs"] == 20].iloc[0]
        alt  = group[group["epochs"] == 50].iloc[0]
        for target in ["top1_acc_best", "runtime_min"]:
            diff_abs = alt[target] - base[target]
            diff_rel = diff_abs / base[target] * 100
            comparisons.append({
                "param": "epochs",
                "control": f"img={img}, frac={frac}",
                "from→to": "20→50",
                "target": target,
                "abs_change": round(diff_abs, 4),
                "rel_change_%": round(diff_rel, 2)
            })

# Img_size effect (control epochs + fraction)
for (ep, frac), group in df.groupby(["epochs", "fraction"]):
    if {224, 512}.issubset(set(group["img_size"])):
        base = group[group["img_size"] == 224].iloc[0]
        alt  = group[group["img_size"] == 512].iloc[0]
        for target in ["top1_acc_best", "runtime_min"]:
            diff_abs = alt[target] - base[target]
            diff_rel = diff_abs / base[target] * 100
            comparisons.append({
                "param": "img_size",
                "control": f"epochs={ep}, frac={frac}",
                "from→to": "224→512",
                "target": target,
                "abs_change": round(diff_abs, 4),
                "rel_change_%": round(diff_rel, 2)
            })

# Fraction effect (control epochs + img_size)
for (ep, img), group in df.groupby(["epochs", "img_size"]):
    if {0.1, 1.0}.issubset(set(group["fraction"])):
        base = group[group["fraction"] == 0.1].iloc[0]
        alt  = group[group["fraction"] == 1.0].iloc[0]
        for target in ["top1_acc_best", "runtime_min"]:
            diff_abs = alt[target] - base[target]
            diff_rel = diff_abs / base[target] * 100
            comparisons.append({
                "param": "fraction",
                "control": f"epochs={ep}, img={img}",
                "from→to": "0.1→1.0",
                "target": target,
                "abs_change": round(diff_abs, 4),
                "rel_change_%": round(diff_rel, 2)
            })

comp_df = pd.DataFrame(comparisons)
comp_df = comp_df.sort_values(by=["param", "target", "control"]).reset_index(drop=True)
summary = (
    comp_df
    .groupby(["param", "target", "from→to"], as_index=False)[["abs_change", "rel_change_%"]]
    .mean()
    .round(4)
)
summary.sort_values(by=["target", "param"]).reset_index(drop=True)

,param,target,from→to,abs_change,rel_change_%
0,epochs,runtime_min,20→50,32.0510,146.1300
1,fraction,runtime_min,0.1→1.0,70.6713,406.4600
2,img_size,runtime_min,224→512,69.9337,401.2333
3,epochs,top1_acc_best,20→50,0.1560,44.9350
4,fraction,top1_acc_best,0.1→1.0,0.2034,52.7075
5,img_size,top1_acc_best,224→512,0.1467,42.7975


### Predict custom uploaded images

In [11]:
upload_dir = Path(DATASET_YAML) / "upload"
models = list(PROJECT_PATH.glob("food101_*/weights/best.pt"))
print(f"Found {len(models)} models")

IMAGE_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

for m in models:
    exp_name = m.parent.parent.name  # e.g. food101_e20_img224_frac0.1
    print(f"▶️ Predicting with {exp_name}")

    # parse imgsz from name (expects "..._img{N}_...")
    try:
        img_size = int(exp_name.split("_img")[1].split("_")[0])
    except Exception:
        img_size = 224

    model = YOLO(m)
    results = model.predict(
        source=upload_dir,
        imgsz=img_size,
        device="cpu",
        save=True,              # writes labeled images into project/name/
        project=PROJECT_PATH,
        name=f"predict_{exp_name}",
        exist_ok=True,
        verbose=False
    )

    # sort the **labeled** images into class folders
    sorted_dir = PROJECT_PATH / f"sorted_{exp_name}"
    sorted_dir.mkdir(parents=True, exist_ok=True)

    for r in results:
        pred_class = r.names[r.probs.top1]

        # where Ultralytics saved this labeled image
        save_dir = Path(r.save_dir)                # predict folder for this run
        src_name = Path(r.path).name               # original filename
        labeled_path = save_dir / src_name         # typical case (same name)

        if not labeled_path.exists():
            # fallback: find any image starting with the same stem (covers suffix variants)
            stem = Path(src_name).stem
            cand = []
            for ext in IMAGE_EXTS:
                cand += list(save_dir.glob(f"{stem}*{ext}"))
            labeled_path = cand[0] if cand else None

        # target folder by predicted class
        target_dir = sorted_dir / pred_class
        target_dir.mkdir(parents=True, exist_ok=True)

        if labeled_path and labeled_path.exists():
            shutil.copy(labeled_path, target_dir / labeled_path.name)
        else:
            # fallback: copy original if labeled not found
            shutil.copy(r.path, target_dir / src_name)

    print(f"✅ Labeled predictions sorted to {sorted_dir}")


Found 8 models
▶️ Predicting with food101_e20_img224_frac0.1
Results saved to /workspaces/marktguru-home-assignment/runs/classify/predict_food101_e20_img224_frac0.1
✅ Labeled predictions sorted to ../runs/classify/sorted_food101_e20_img224_frac0.1
▶️ Predicting with food101_e50_img512_frac1
Results saved to /workspaces/marktguru-home-assignment/runs/classify/predict_food101_e50_img512_frac1
✅ Labeled predictions sorted to ../runs/classify/sorted_food101_e50_img512_frac1
▶️ Predicting with food101_e50_img224_frac1
Results saved to /workspaces/marktguru-home-assignment/runs/classify/predict_food101_e50_img224_frac1
✅ Labeled predictions sorted to ../runs/classify/sorted_food101_e50_img224_frac1
▶️ Predicting with food101_e50_img512_frac0.1
Results saved to /workspaces/marktguru-home-assignment/runs/classify/predict_food101_e50_img512_frac0.1
✅ Labeled predictions sorted to ../runs/classify/sorted_food101_e50_img512_frac0.1
▶️ Predicting with food101_e50_img224_frac0.1
Results saved to /w